# RGB Image classification

This tutorial shows how to classify images into 3 categories either Red, Green or Blue. It builds an image classifier using a `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`. 

* Building _data input pipelines_ using the `tf.keras.preprocessing.image.ImageDataGenerator` class to efficiently work with data on disk to use with the model.
* _Overfitting_ —How to identify and prevent it.
* _Data augmentation_ and _dropout_ —Key techniques to fight overfitting in computer vision tasks to incorporate into the data pipeline and image classifier model.

This notebook follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import packages

We start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

Import Tensorflow and the Keras classes needed to construct our model.

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

## Load data

Begin by downloading the dataset. Here I am generating mono-solid colored image with the help of color_img function.

The dataset has the following directory structure:

<pre>
<b>dataset/b>
|__ <b>train</b>
    |______ <b>red_dir</b>: [red-0.jpg, red-1.jpg, red-2.jpg ....]
    |______ <b>blue_dir</b>: [blue-0.jpg, blue-1.jpg, blue-2.jpg ...]
     |______ <b>green_dir</b>: [green-0.jpg, green-1.jpg, green-2.jpg ...]
|__ <b>validation</b>
   |______ <b>red_dir</b>: [red-20.jpg, red-21.jpg, red-22.jpg ....]
    |______ <b>blue_dir</b>: [blue-20.jpg, blue-21.jpg, blue-22.jpg ...]
     |______ <b>green_dir</b>: [green-20.jpg, green-21.jpg, green-22.jpg ...]
</pre>

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#print(os.path)
!mkdir /content/train/
!mkdir /content/train/red_dir
!mkdir /content/train/green_dir
!mkdir /content/train/blue_dir
!mkdir /content/validation/
!mkdir /content/validation/red_dir
!mkdir /content/validation/green_dir
!mkdir /content/validation/blue_dir

In [0]:
index = 0
def strim_string(string):
    
    string = string.replace(' ', '')
    string = string.replace('\\n', '')
    string = string.replace('\\t', '')

    return string

def create_color_image(height,
                       width,
                       red_min,
                       red_max,
                       green_min,
                       green_max,
                       blue_min,
                       blue_max,
                       num_images,
                       name,
                       path = None):
    
    """
    We provide images in BGR for OPENCV
    """
    global index
    path='/content'
    img = np.array([0 for i in range(height*width*3)]).reshape(height,width,3).astype(np.uint8)
    
    img[:,:,2] = red_min # Red
    img[:,:,1] = green_min # Green
    img[:,:,0] = blue_min # Blue
    
    i = (red_max - red_min) // num_images
    j = (green_max - green_min) // num_images
    k = (blue_max - blue_min) // num_images
            
    for _ in range(0, num_images, 1):
        
        img[:,:,2] += i # Red
        img[:,:,1] += j # Green
        img[:,:,0] += k # Blue
        
        save_name = '{}-{}.jpg'.format(name, index)   
        save_path =f'/content/train/{name}_dir/'
        img_path = save_path + save_name

        # while os.path.exists(save_path):
        #     index += 1
        #     save_name = '{}-{}.jpg'.format(name, index)   
        #     save_path = save_name
            
        #     if path:
        #         save_path = path + '/' + save_name
        try:        
          cv2.imwrite(img_path, img)
          index += 1
          print(str(index)+'done' +' '+img_path)
        except:
          print('Errors')

In [0]:
def main():
    
    colors = pd.read_csv('/content/colors.csv', header = None)
    
    for line in range(colors.shape[0]):
    
        lst = colors.iloc[line,:]
        lst[6] = strim_string(lst[6])
        
        create_color_image(15, 15,int(lst[0]), int(lst[1]),int(lst[2]), int(lst[3]), int(lst[4]), int(lst[5]),20,lst[6])

In [0]:
main()

#Training

In [0]:
# import tensorflow as tf
# import numpy as np

# class CNN(object):
    
#     def __init__(self, 
#                  batch_size,
#                  learning_rate,
#                  shape,
#                  num_classes):
        
#         self.batch_size = batch_size
#         self.learning_rate = learning_rate
#         self.shape = shape
#         self.num_classes = num_classes
        
#         self.strides_size = 2
#         self.kernel_size = 2
#         self.padding = 'SAME'
        
#     def init_weights(self, shape):
#         random_dist = tf.truncated_normal(shape, stddev = 0.1)
#         return tf.Variable(random_dist)
    
#     def init_bias(self, shape):
#         bias = tf.constant(0.1, shape = [shape])
#         return tf.Variable(bias)
        
#     def conv_layer(self, 
#                    x, 
#                    shape,
#                    name = 'conv_'):
        
#         W = self.init_weights(shape)
#         b = self.init_bias(shape[3])
        
#         conv = tf.nn.conv2d(x, W, strides=[1, self.strides_size, self.strides_size, 1],
#                             padding = self.padding)
#         layer = tf.add(conv, b)
#         act = tf.nn.relu(layer)
        
#         return act
        
#     def fc_layer(self, X, shape):
        
#         input_size = int(X.get_shape()[1])
#         W = self.init_weights([input_size, shape])
#         b = self.init_bias(shape)
        
#         return tf.matmul(X, W ) + b
    
#     def neural_net(self, X, hold_prob = 0.5):
        
#         with tf.variable_scope('convolution') as scope:
            
#             conv1 = self.conv_layer(X, [3, 3, 3,  32])
#             conv2 = self.conv_layer(conv1, [5,5, 32, 64])            
            
#         with tf.variable_scope('pooling') as scope:
            
#             pool1 = tf.nn.max_pool(conv2, 
#                                    ksize = [1, self.kernel_size , self.kernel_size ,1], 
#                                    strides=[1, self.strides_size, self.strides_size , 1],
#                                    padding = 'VALID')
            
         
#         with tf.variable_scope('fc_layer') as scope:
            
#             flat = tf.reshape(pool1, [-1 , 8 * 8 * 64])
#             dropout_flat = tf.nn.dropout(flat, keep_prob=hold_prob) 
            
#             fc1 = self.fc_layer(dropout_flat, 1024)
#             fc1 = tf.nn.relu(fc1)
#             dropout_fc1 = tf.nn.dropout(fc1, keep_prob=hold_prob)
            
#             fc2 = self.fc_layer(dropout_fc1, self.num_classes)
#             #fc2 = tf.nn.relu(fc2)
         
#         self.y_prob = fc2
#         #self.y_prob = tf.nn.softmax(fc2, name = 'Y_prob')
            
#     def train_model(self, y,):
#         pass

After extracting its contents, assign variables with the proper file path for the training and validation set.

In [0]:
train_dir = '/content/train'
validation_dir = '/content/validation'

In [0]:
train_red_dir = train_dir+'/red_dir' # directory with our training red pictures
train_green_dir = train_dir+'/green_dir'  # directory with our training red pictures
train_blue_dir = train_dir+'/blue_dir'  # directory with our training blue pictures
validation_red_dir = validation_dir+'/red_dir'  # directory with our validation red pictures
validation_green_dir = validation_dir+'/green_dir'  # directory with our validation green pictures
validation_blue_dir = validation_dir+'/blue_dir'  # directory with our validation blue pictures


### Understand the data

Let's look at how many Red, green and blue images are in the training and validation directory:

In [0]:
num_red_tr = len(os.listdir(train_red_dir))

For convenience, set up variables to use while pre-processing the dataset and training the network.

In [0]:
batch_size = 10
epochs = 15
IMG_HEIGHT = 15
IMG_WIDTH = 15

## Data preparation

Format the images into appropriately pre-processed floating point tensors before feeding to the network:

1. Read images from the disk.
2. Decode contents of these images and convert it into proper grid format as per their RGB content.
3. Convert them into floating point tensors.
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.

Fortunately, all these tasks can be done with the `ImageDataGenerator` class provided by `tf.keras`. It can read images from disk and preprocess them into proper tensors. It will also set up generators that convert these images into batches of tensors—helpful when training the network.

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which is 32 images in this example—then plot five of them with `matplotlib`.

In [0]:
sample_training_images, _ = next(train_data_gen)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(5,5))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
plotImages(sample_training_images[:5])

## Create the model

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 512 units on top of it that is activated by a `relu` activation function.



keras.layers.Dense(2, activation = 'softmax')(previousLayer)
Usually, we use the softmax activation function to do classification tasks, and the output width will be the number of the categories. This means that if you want to classify one object into three categories with the labels A,B, or C, you would need to make the Dense layer generate an output with a shape of (None, 3). Then you can use the cross_entropyloss function to calculate the LOSS, automatically calculate the gradient, and do the back-propagation process.

If you want to only generate one value with the Dense layer, that means you get a tensor with a shape of (None, 1) - so it produces a single numeric value, like a regression task. You are using the value of the output to represent the category. The answer is correct, but does not perform like the general solution of the classification task.

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Compile the model

For this tutorial, choose the *ADAM* optimizer and *binary cross entropy* loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Model summary

View all the layers of the network using the model's `summary` method:

In [0]:
model.summary()

### Train the model

Use the `fit_generator` method of the `ImageDataGenerator` class to train the network.

In [0]:
total_train = 180
total_val = 90

In [0]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Visualize training results

Now visualize the results after training the network.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

As you can see from the plots, training accuracy and validation accuracy are off by large margin and the model has achieved only around **70%** accuracy on the validation set.

Let's look at what went wrong and try to increase overall performance of the model.

## Overfitting

In the plots above, the training accuracy is increasing linearly over time, whereas validation accuracy stalls around 70% in the training process. Also, the difference in accuracy between training and validation accuracy is noticeable—a sign of *overfitting*.

When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples—to an extent that it negatively impacts the performance of the model on new examples. This phenomenon is known as overfitting. It means that the model will have a difficult time generalizing on a new dataset.

There are multiple ways to fight overfitting in the training process. In this tutorial, you'll use *data augmentation* and add *dropout* to our model.

## Data augmentation

Overfitting generally occurs when there are a small number of training examples. One way to fix this problem is to augment the dataset so that it has a sufficient number of training examples. Data augmentation takes the approach of generating more training data from existing training samples by augmenting the samples using random transformations that yield believable-looking images. The goal is the model will never see the exact same picture twice during training. This helps expose the model to more aspects of the data and generalize better.

Implement this in `tf.keras` using the `ImageDataGenerator` class. Pass  different transformations to the dataset and it will take care of applying it during the training process.

### Augment and visualize data

Begin by applying random horizontal flip augmentation to the dataset and see how individual images look like after the transformation.

### Apply horizontal flip

Pass `horizontal_flip` as an argument to the `ImageDataGenerator` class and set it to `True` to apply this augmentation.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Take one sample image from the training examples and repeat it five times so that the augmentation is applied to the same image five times.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
# Re-use the same custom plotting function defined and used
# above to visualize the training images
plotImages(augmented_images)

### Randomly rotate the image

Let's take a look at a different augmentation called rotation and apply 45 degrees of rotation randomly to the training examples.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Apply zoom augmentation

Apply a zoom augmentation to the dataset to zoom images up to 50% randomly.

In [0]:
# zoom_range from 0 - 1 where 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Put it all together

Apply all the previous augmentations. Here, you applied rescale, 45 degree rotation, width shift, height shift, horizontal flip and zoom augmentation to the training images.

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

Visualize how a single image would look five different times when passing these augmentations randomly to the dataset.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Create validation data generator

Generally, only apply data augmentation to the training examples. In this case, only rescale the validation images and convert them into batches using `ImageDataGenerator`.

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [0]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Dropout

Another technique to reduce overfitting is to introduce *dropout* to the network. It is a form of *regularization* that forces the weights in the network to take only small values, which makes the distribution of weight values more regular and the network can reduce overfitting on small training examples. Dropout is one of the regularization technique used in this tutorial

When you apply dropout to a layer it randomly drops out (set to zero) number of output units from the applied layer during the training process. Dropout takes a fractional number as its input value, in the form such as 0.1, 0.2, 0.4, etc. This means dropping out 10%, 20% or 40% of the output units randomly from the applied layer.

When appling 0.1 dropout to a certain layer, it randomly kills 10% of the output units in each training epoch.

Create a network architecture with this new dropout feature and apply it to different convolutions and fully-connected layers.

## Creating a new network with Dropouts

Here, you apply dropout to first and last max pool layers. Applying dropout will randomly set 20% of the neurons to zero during each training epoch. This helps to avoid overfitting on the training dataset.

In [0]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3)
])

### Compile the model

After introducing dropouts to the network, compile the model and view the layers summary.

In [0]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

### Train the model

After successfully introducing data augmentations to the training examples and adding dropouts to the network, train this new network:

In [0]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Visualize the model

Visualize the new model after training, you can see that there is significantly less overfitting than before. The accuracy should go up after training the model for more epochs.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()